## 简答题
1. 如果训练集有100万个实例，1.训练决策树（无约束）大致的深度是多少？
大约 log₂(1 000 000) ≈ 20 层叶子。

2. 通常来说，子节点的基尼杂质是高于还是低于其父节点？是通常更高/更低？还是永远更高/更低？
CART 每次分裂都会选使加权基尼杂质之和最小的分割点，因此子节点的加权平均基尼杂质一定小于（或等于）父节点。

3. 如果决策树过拟合训练集，减少max_depth是否为一个好主意？
如果决策树过拟合训练集，减少max_depth确实可以。

4. 如果决策树对训练集欠拟合，尝试缩放输入特征是否为一个好主意？
决策树对特征的尺度不敏感，欠拟合应尝试减小正则化。

5. 如果在给定的训练集上训练决策树需要一个小时，那么如果将特征数量变为两倍，训练大约需要多少时间？
训练时间大约也变为 2 倍。

6. 如果在包含100万个实例的训练集上训练决策树需要一个小时，那么在包含1000万个实例的训练集上训练决策树，大概需要多长时间？提示：考虑CART算法的计算复杂度。
CART 算法在大规模数据下的复杂度大约是 O(n log n)  ，所以为10log10 =10，因而训练时间大致变为 10 小时 左右。

## 编程题

1. 为 新月形 数据集训练并微调一棵决策树。

a. 使用make_moons(n_samples=10000，noise=0.4)生成一个 新月形 数据集。

b. 使用train_test_split()拆分训练集和测试集。

c. 使用交叉验证的网格搜索（在GridSearchCV类的帮助下）为Decision-TreeClassifier找到适合的超参数值。提示：尝试max_leaf_nodes的多种值。

d. 使用超参数对整个训练集进行训练，并测量模型在测试集上的性能。你应该得到约85%～87%的准确率。

2. 按照以下步骤种植森林。

a.继续之前的练习，生产1000个训练集子集，每个子集包含随机挑选的100个实例。提示：使用Scikit-Learn的ShuffleSplit类来实现。

b.使用前面得到的最佳超参数值，在每个子集上训练一棵决策树。在测试集上评估这1000棵决策树。因为训练集更小，所以这些决策树的表现可能比第一棵决策树要差一些，只能达到约80%的精度。

c.见证奇迹的时刻到了。用每个测试集实例，生成1000棵决策树的预测，然后仅保留次数最频繁的预测［可以使用SciPy的mode()函数］。这样你在测试集上可获得大多数投票的预测结果。

d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高（高出0.5%～1.5%）。恭喜，你已经训练出了一个随机森林分类器！

In [ ]:
# 为 新月形 数据集训练并微调一棵决策树

In [6]:
from sklearn.datasets import make_moons
from tensorflow.python.ops.math_ops import linspace

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [7]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.model_selection import cross_val_score


model_decisiontree = DecisionTreeClassifier(max_depth=None,random_state=42)


params = {
    'max_leaf_nodes': np.arange(2, 100),
    'min_samples_split': np.arange(2, 5),
}

rand = RandomizedSearchCV(
    model_decisiontree,
    params,
    n_iter=50,
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

rand.fit(X_train, y_train)
best_tree = rand.best_estimator_
y_pred = best_tree.predict(X_test)

final_scores =cross_val_score(best_tree, X_test, y_test, cv=5)
print(f"准确率: {final_scores.mean():.4f} ± {final_scores.std():.4f}")
print(f"最佳参数: {rand.best_params_}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits
准确率: 0.8525 ± 0.0079
最佳参数: {'min_samples_split': 2, 'max_leaf_nodes': 23}


In [28]:
# 用SVM法
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


model_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf'))
])

params = {
    'svc__gamma': np.linspace(0.5,5.0,10),
    'svc__C':  np.linspace(1,20,20),
}

grid = GridSearchCV(
    model_svm,
    params,
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)
best_models = grid.best_estimator_
y_pred = best_models.predict(X_test)

final_scores1 =cross_val_score(best_models, X_test, y_test, cv=5)
print(f"准确率: {final_scores1.mean():.4f} ± {final_scores1.std():.4f}")
print(f"最佳参数: {grid.best_params_}")

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
准确率: 0.8660 ± 0.0183
最佳参数: {'svc__C': 18.0, 'svc__gamma': 2.0}


In [25]:
# 用 Bagging
base_tree = DecisionTreeClassifier(max_depth=None, random_state=42)

bagging = BaggingClassifier(
    estimator=base_tree,      # 基学习器
    n_estimators=500,         # 树的数量
    max_samples=0.8,          # 每棵树用 80% 样本做自助采样
    max_features=1.0,         # 每棵树用全部特征
    n_jobs=-1,
    random_state=42
)

scores = cross_val_score(bagging, X, y, cv=5, n_jobs=-1)
print(f"Bagging + 决策树准确率: {scores.mean():.4f} ± {scores.std():.4f}")


Bagging + 决策树准确率: 0.8452 ± 0.0050


In [ ]:
#按照以下步骤种植森林。

In [29]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [31]:
from sklearn.base import clone

forest = [clone(rand.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.79884

In [32]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)


In [33]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8735